In [30]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn


In [31]:
read_data = pd.read_csv('model.csv')
read_test = pd.read_csv('predict_test.csv')

In [32]:
read_data = read_data.drop('Unnamed: 0', axis = 1)
read_test = read_test.drop('Unnamed: 0', axis = 1)

In [33]:
read_data.shape, read_test.shape

((95662, 21), (45019, 20))

In [34]:
y = read_data['FraudResult']
x = read_data.drop('FraudResult', axis = 1)

In [35]:
sum(y)/y.shape[0], sum(y), y.shape[0]

(0.00201752001839811, 193, 95662)

In [36]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
min_max = MinMaxScaler()

train_split =std.fit_transform(x)

test_data = std.fit_transform(read_test)

In [37]:
train_split = min_max.fit_transform(train_split)
test_data = min_max.fit_transform(test_data)

In [39]:
from sklearn.feature_selection import SelectKBest, chi2

X_new = SelectKBest(chi2, 10).fit_transform(train_split, y)

In [9]:
from sklearn.model_selection import train_test_split
y = y.astype('int')
x_train, x_test,y_train, y_test = train_test_split(X_new, y, train_size = 0.8, random_state = 42)


In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
from sklearn.metrics import accuracy_score, precision_score, f1_score
lr.fit(x_train, y_train)
lr.score(x_train, y_train), lr.score(x_test, y_test)

(0.9982359628376171, 0.9983797627136362)

In [11]:
pred = lr.predict(x_test)

f1_score(pred, y_test), precision_score(pred, y_test)

(0.24390243902439027, 0.1388888888888889)

In [12]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)



Using TensorFlow backend.


In [13]:
x_train_res.shape

(152744, 20)

In [14]:
test_data.shape

(45019, 20)

In [15]:
lr.fit(x_train_res, y_train_res)
lr.score(x_train_res, y_train_res)

0.9835083538469597

In [16]:
pred = lr.predict(x_test)
f1_score(pred, y_test)

0.18701298701298702

In [17]:
precision_score(pred, y_test)

1.0

In [18]:

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train_res, y_train_res)
pred = knn.predict(x_test)
f1_score(pred, y_test)

0.7441860465116279

In [ ]:
dict_score = {}
for k in [1,2,3,4,5,6,7,8,9,10]:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(x_train_res, y_train_res)
    pred = knn.predict(x_test)
    score = f1_score(pred, y_test)
    dict_score[str(k)] = score

dict_score

In [19]:
from sklearn.ensemble import RandomForestClassifier
rand = RandomForestClassifier()
rand.fit(x_train_res, y_train_res)
pred = rand.predict(x_test)
f1_score(pred, y_test)

0.923076923076923

In [20]:
rand.score(x_train_res, y_train_res)

0.9998821557638925

In [21]:
accuracy_score(pred, y_test), precision_score(pred, y_test)

(0.9996864056865102, 1.0)

In [27]:
prediction = knn.predict(read_test)

In [23]:
ids = pd.read_csv('TestId.csv')
ids.head()

,Unnamed: 0,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProductId,ChannelId
0,0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,ProductId_3,ChannelId_3
1,1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,ProductId_15,ChannelId_3
2,2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,ProductId_6,ChannelId_2
3,3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,ProductId_10,ChannelId_3
4,4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,ProductId_6,ChannelId_2


In [28]:
test = pd.DataFrame()
test['TransactionId'] = ids['TransactionId']
test['FraudResult'] =prediction.astype('int')

test.to_csv('rand.csv', index = False)

In [29]:
test.shape

(45019, 2)